<center><h1>Implementación de un Modelo de ML. Dataset a utilizar</h1></center>

Usaremos el siguiente dataset encontrado en Kaggle, en el siguiente link: </br>
https://www.kaggle.com/datasets/mfarhaannazirkhan/heart-dataset

Se ha seleccionado este dataset, por estar más actualizado e incluir referencias

Acorde con la descripción del dataset tendremos las siguientes columnas:


1. age: Age of the patient (Numeric).
2. sex: Gender of the patient. Values: 1 = male, 0 = female.
3.     cp: Chest pain type. Values: 0 = Typical angina, 1 = Atypical angina, 2 = Non-anginal pain, 3 = Asymptomatic.
4. trestbps: Resting Blood Pressure (in mm Hg) (Numeric).
5. chol: Serum Cholesterol level (in mg/dl) (Numeric).
6. fbs: Fasting blood sugar > 120 mg/dl. Values: 1 = true, 0 = false.
7. restecg: Resting electrocardiographic results. Values: 0 = Normal, 1 = ST-T wave abnormality, 2 = Left ventricular hypertrophy.
8. thalach: Maximum heart rate achieved (Numeric).
9. exang: Exercise-induced angina. Values: 1 = yes, 0 = no.
10. oldpeak: ST depression induced by exercise relative to rest (Numeric).
11. slope: Slope of the peak exercise ST segment. Values: 0 = Upsloping, 1 = Flat, 2 = Downsloping.
12. ca: Number of major vessels (0-3) colored by fluoroscopy. Values: 0, 1, 2, 3.
13. thal: Thalassemia types. Values: 1 = Normal, 2 = Fixed defect, 3 = Reversible defect.
14. target: Outcome variable (heart attack risk). Values: 1 = more chance of heart attack, 0 = less chance of heart attack.

In [ ]:
#cargamos el dataset
import pandas as pd

df =pd.read_csv('cleaned_merged_heart_dataset.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalachh,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
#partimos en entrenamiento y test
from sklearn.model_selection import train_test_split

#definir variables predictoras
X = df.drop("target", axis=1)
y = df["target"]

#dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("filas entrenamiento", X_train.shape[0])
print("filas prueba", X_test.shape[0])

filas entrenamiento 1510
filas prueba 378


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.ensemble import RandomForestClassifier

# Métricas a evaluar
metricas = ['accuracy', 'precision', 'recall', 'f1']

# Definir modelo
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Definir folds estratificados
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

# Validación cruzada directamente
scores_rf = cross_validate(
    rf, X_train, y_train,
    scoring=metricas, cv=cv, n_jobs=-1, return_train_score=False
)

# Resultados promedio y desviación
print("Resultados CV (5-fold) - Random Forest")
for m in metricas:
    vals = scores_rf[f"test_{m}"]
    print(f"{m:>9}: {vals.mean():.3f} ± {vals.std():.3f}")


Resultados CV (5-fold) - Random Forest
 accuracy: 0.968 ± 0.007
precision: 0.969 ± 0.007
   recall: 0.969 ± 0.007
       f1: 0.969 ± 0.007


In [ ]:
# 1) Entrenar el modelo final con TODO X_train, y_train
rf.fit(X_train, y_train)

# 2) Evaluar en el conjunto de prueba
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]  # si quieres AUC u umbrales

print("Matriz de confusión")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de clasificación")
print(classification_report(y_test, y_pred))

# (opcional) AUC si es problema binario
try:
    print("ROC AUC:", roc_auc_score(y_test, y_proba))
except Exception as e:
    pass


Matriz de confusión
[[177   5]
 [  1 195]]

Reporte de clasificación
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       182
           1       0.97      0.99      0.98       196

    accuracy                           0.98       378
   macro avg       0.98      0.98      0.98       378
weighted avg       0.98      0.98      0.98       378

ROC AUC: 0.9987805561785154


# Vamos a pasar el modelo a producción

In [ ]:
#importamos joblib
import joblib

# 2) Guardar modelo entrenado
joblib.dump(rf, "random_forest_model.joblib")

['random_forest_model.joblib']

In [ ]:
#Exploremos un poco ese archivo
import numpy as np
import pandas as pd

# Cargar el modelo entrenado
rf = joblib.load('random_forest_model.joblib')

# Atributos clave del RandomForest entrenado
print(type(rf).__name__)
print('n_estimators', rf.n_estimators)
print('classes:', rf.classes_)
print('n_feauteres', rf.n_features_in_)
print('nombres', rf.get_params())

RandomForestClassifier
n_estimators 100
classes: [0 1]
n_feauteres 13
nombres {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


### Interpretación rápida de  resultados

- **thal (14,42%)** – Tipo de talasemia del paciente; parece ser la variable más influyente para predecir riesgo de ataque cardíaco.  
- **cp (14,22%)** – Tipo de dolor de pecho; muy relevante para el diagnóstico.  
- **thalachh (10,89%)** – Máxima frecuencia cardíaca alcanzada; indica la capacidad de esfuerzo.  
- **oldpeak (9,14%)** – Depresión del segmento ST; importante en electrocardiogramas.  
- **ca (8,47%)** – Número de vasos mayores visualizados en fluoroscopia.  
- **chol (8,23%)** – Nivel de colesterol.  
- **age (8,14%)** – Edad del paciente.  
- **slope (8,07%)** – Pendiente del segmento ST.  
- **trestbps (7,44%)** – Presión arterial en reposo.  
- **restecg (3,60%)** – Resultados del electrocardiograma en reposo.  


In [ ]:
# Si aún tienes X_train a mano:
feature_names = list(X_train.columns)
importances = rf.feature_importances_
top_idx = np.argsort(importances)[::-1][:10]

print("\nTop 10 features:")
for i in top_idx:
    print(f"{feature_names[i]:<20} {importances[i]:.4f}")



Top 10 features:
thal                 0.1442
cp                   0.1422
thalachh             0.1089
oldpeak              0.0914
ca                   0.0847
chol                 0.0823
age                  0.0814
slope                0.0807
trestbps             0.0744
restecg              0.0360


In [ ]:
#hagamos una sola predicción


import pandas as pd

# 1. Cargar el modelo entrenado
rf_joblib = joblib.load("random_forest_model.joblib")

# 2. Crear un DataFrame con la observación que quieres predecir
# Debe tener las mismas columnas y en el mismo orden que X_train
nueva_muestra = pd.DataFrame([{
    "age": 63,
    "sex": 1,
    "cp": 3,
    "trestbps": 145,
    "chol": 233,
    "fbs": 1,
    "restecg": 0,
    "thalachh": 150,
    "exang": 0,
    "oldpeak": 2.3,
    "slope": 0,
    "ca": 0,
    "thal": 1
}])

# 3. Hacer la predicción
prediccion = rf_joblib.predict(nueva_muestra)[0]
probabilidad = rf_joblib.predict_proba(nueva_muestra)[0][1]

print(f"Predicción: {prediccion} (1 = más riesgo, 0 = menos riesgo)")
print(f"Probabilidad de riesgo: {probabilidad:.2f}")


Predicción: 1 (1 = más riesgo, 0 = menos riesgo)
Probabilidad de riesgo: 0.99


# Creando un servicio web

In [ ]:
#esta sección descarga las dependencias para crear un servidor web local
#sin embargo el entorno de colab es aislado, por lo cual usaremos un servicio
#llamado ngrok que crea un tunel desde el servicio web local, hacia el exterior
!pip -q install pyngrok fastapi uvicorn


In [ ]:
#debemos registrarnos en https://ngrok.com/
#hasta la fecha (Agosto 2025) no es necesario registrar tarjeta de crédito o débito
#Vamos a obtener un token de autenticación que podremos pegar acá
import getpass

from pyngrok import ngrok, conf

# 1) Authtoken (no lo pegues en claro)
from getpass import getpass
NGROK_TOKEN = getpass("Pega tu ngrok authtoken: ")

from pyngrok import ngrok
ngrok.set_auth_token(NGROK_TOKEN)

Pega tu ngrok authtoken: ··········


In [ ]:
#esta sección cierra túneles  de salida para la aplicación
#como máximo podemos crear 3 tuneles (aunque con uno es suficiente)
#pero cada vez que ejecutamos la  celda hacia el final, se puede crear uno, pasando el límite
#por lo cual se agrega código para cerrar dichos túneles
from pyngrok import ngrok

# Ver túneles activos (opcional)
for t in ngrok.get_tunnels():
    print("Activo:", t.public_url, "->", t.config.get("addr"))

# Cerrar TODOS los túneles
for t in ngrok.get_tunnels():
    try:
        ngrok.disconnect(t.public_url)
    except Exception as e:
        print("No se pudo cerrar", t.public_url, e)

# Matar el proceso del agente
ngrok.kill()


In [ ]:
#esta sección crea la página web que va a ser expuesta
#dicha página carga el modelo entrenado random_forest_model.joblib

%%writefile main.py
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse, JSONResponse
import pandas as pd
import joblib

app = FastAPI()
model = joblib.load("random_forest_model.joblib")

FEATURES = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

@app.get("/", response_class=HTMLResponse)
def form():
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Heart Risk Prediction</title>
        <style>
            body { font-family: Arial; max-width: 800px; margin: 40px auto }
            input, select { width: 160px; margin: 4px }
            .row { display: grid; grid-template-columns: repeat(3, 1fr); gap: 12px; margin-bottom: 20px }
            button { padding: 10px 20px }
            #output { margin-top: 20px; font-size: 16px; font-weight: bold }
        </style>
    </head>
    <body>
        <h2>Heart Disease Risk Prediction</h2>
        <div class="row">
        <div><label>age</label><br><input type="number" id="age" step="any" value="63"></div><div><label>Gender</label><br><select id="sex"><option value="1" selected>Male</option><option value="0">Female</option></select></div><div><label>Chest Pain Type</label><br><select id="cp"><option value="0">Typical angina</option><option value="1">Atypical angina</option><option value="2">Non-anginal pain</option><option value="3" selected>Asymptomatic</option></select></div><div><label>trestbps</label><br><input type="number" id="trestbps" step="any" value="145"></div><div><label>chol</label><br><input type="number" id="chol" step="any" value="233"></div><div><label>Fasting Blood Sugar > 120 mg/dl</label><br><select id="fbs"><option value="1" selected>True</option><option value="0">False</option></select></div><div><label>Resting ECG Results</label><br><select id="restecg"><option value="0" selected>Normal</option><option value="1">ST-T wave abnormality</option><option value="2">Left ventricular hypertrophy</option></select></div><div><label>thalachh</label><br><input type="number" id="thalachh" step="any" value="150"></div><div><label>Exercise-Induced Angina</label><br><select id="exang"><option value="1">Yes</option><option value="0" selected>No</option></select></div><div><label>oldpeak</label><br><input type="number" id="oldpeak" step="any" value="2.3"></div><div><label>Slope of the Peak ST Segment</label><br><select id="slope"><option value="0" selected>Upsloping</option><option value="1">Flat</option><option value="2">Downsloping</option></select></div><div><label>Major Vessels Colored by Fluoroscopy</label><br><select id="ca"><option value="0" selected>0</option><option value="1">1</option><option value="2">2</option><option value="3">3</option></select></div><div><label>Thalassemia Type</label><br><select id="thal"><option value="1" selected>Normal</option><option value="2">Fixed defect</option><option value="3">Reversible defect</option></select></div>
        </div>
        <button onclick="enviar()">Predecir</button>
        <div id="output"></div>
        <script>
        async function enviar() {
            const data = {};
            const campos = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exang', 'oldpeak', 'slope', 'ca', 'thal'];
            campos.forEach(c => {
                const val = document.getElementById(c).value;
                data[c] = parseFloat(val);
            });
            const res = await fetch("/predict", {
                method: "POST",
                headers: { "Content-Type": "application/json" },
                body: JSON.stringify(data)
            });
            const j = await res.json();
            document.getElementById("output").innerText =
                `Predicción: ${j.prediccion} (1=riesgo) | Probabilidad: ${j.probabilidad_riesgo.toFixed(3)}`;
        }
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=html)

@app.post("/predict")
async def predict(data: dict):
    try:
        df = pd.DataFrame([data])
        df = df[FEATURES]
        pred = int(model.predict(df)[0])
        prob = float(model.predict_proba(df)[0][1])
        return { "prediccion": pred, "probabilidad_riesgo": prob }
    except Exception as e:
        return JSONResponse({ "error": str(e) }, status_code=400)

Writing main.py


In [ ]:
# (Re)inicia el server en segundo plano UNA sola vez
!pkill -f "uvicorn" || true
!nohup uvicorn main:app --host 0.0.0.0 --port 8000 > server.log 2>&1 &
!tail -n 20 server.log


^C


In [ ]:
# Crear túnel HTTP
# En esta celda es dónde realmente creamos el tunel
public_url = ngrok.connect(addr=8000, proto="http")  # <-- clave
print("URL pública:", public_url)


URL pública: NgrokTunnel: "https://78cad4b18e47.ngrok-free.app" -> "http://localhost:8000"
